In [0]:
DEBUG = False

GALAXY = 'm31'
GALAXYFULLNAME = 'm31'

CONFIG_FILE = f'../../../configs/netflow/{GALAXYFULLNAME}.py'
NVISITS = 1

# Plot the target lists

Load the target lists and plot them.

In [0]:
import os, sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', int(os.environ['PFS_TARGETING_DEBUGPORT'])))
    debug = True

# Plot definitions

In [0]:
import pfs.utils
from pfs.ga.targeting.targets.m31 import *
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import WcsProjection, Pointing
from pfs.ga.targeting.netflow import Netflow
from pfs.ga.targeting.io import DataFrameSerializer

In [0]:
galaxy = GALAXIES[GALAXY]
hsc = galaxy.get_photometry()
cmd = galaxy.get_cmd()
ccd = galaxy.get_ccd()
gaia_cmd = galaxy.get_cmd(Gaia)

In [0]:
pointings = galaxy.get_pointings(SubaruPFI)
pointing = pointings[0]

for p in pointings:
    print(p.ra, p.dec, p.posang)

In [0]:
pfi = SubaruPFI(instrument_options={ 'layout': 'calibration' })
wcs = WcsProjection(Pointing(galaxy.pos), proj='TAN')
wfc = SubaruWFC(pointing)
fov = FOV(projection=wcs)
fp = FP(wfc)

# Load the netflow config file

In [0]:
# Get the default configuration for the targets
config = galaxy.get_netflow_config()

# Load the customized configuration on top of the default one
config.load(CONFIG_FILE, ignore_collisions=True)

# Load the target lists

In [0]:
from pfs.ga.targeting.config import NetflowConfig
from pfs.ga.targeting.io import ObservationSerializer
from pfs.ga.targeting.scripts.netflowscript import NetflowScript

In [0]:
# Load the configuration
config = NetflowConfig.from_file(CONFIG_FILE, ignore_collisions=True)

print(config.targets.keys())

In [0]:
target_lists = {}

for k in config.targets:
    target_lists[k] = NetflowScript.load_target_list(k, config.targets[k])
    print(k, config.targets[k].prefix, config.targets[k].path)

In [0]:
cmap = plt.get_cmap('tab10')
colors = { 'sci': 'lightgray', 'cal': 'red', 'sky': cmap(2) }
size = { 'sci': 0.5, 'cal': 2, 'sky': 0.1 }

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

for i, (k, target_list) in enumerate(target_lists.items()):
    if config.targets[k].prefix in ['sci', 'cal']:
        if target_lists[k].has_diagram_values(cmd, observed=True):
            cmd.plot_observation(axs[0], target_list, c=colors[config.targets[k].prefix], observed=True)

for i, (k, target_list) in enumerate(target_lists.items()):
    if config.targets[k].prefix in ['sci', 'cal']:
        if target_lists[k].has_diagram_values(ccd, observed=True):
            ccd.plot_observation(axs[1], target_list, c=colors[config.targets[k].prefix], observed=True)

for i, (k, target_list) in enumerate(target_lists.items()):
    if config.targets[k].prefix in ['sci', 'cal']:
        fov.plot_observation(axs[2], target_list, c=colors[config.targets[k].prefix], size=size[config.targets[k].prefix], observed=True)
        
pp = galaxy.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_aspect('equal', adjustable='datalim')

f.suptitle('Science targets and calibration stars')

In [0]:
# List the priorities classes defined for each science target list
for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        print(k, np.sort(target_lists['dsph'].data['priority'].unique()))

In [0]:
# Plot priority distribution for each target list
for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount(target_lists[k].data['priority'])
        ax.bar(np.arange(hist.size), hist)

        ax.set_title(f'Priority class distribution for target list `{k}`')
        ax.set_xlabel(f'Priority class')
        ax.set_ylabel(f'Target count')

In [0]:
# Plot the distribution of required visits for each target list
exp_time = 1800 # seconds

for k in config.targets:
    if config.targets[k].prefix in ['sci']:
        f, ax = plt.subplots(1, 1, figsize=(3.5, 2.5), dpi=240)

        hist = np.bincount((np.ceil(target_lists[k].data['exp_time'] / exp_time)).astype(int))
        ax.bar(np.arange(hist.size), hist)

        ax.set_title(f'Distribution of required visit (t_exp = {exp_time} s)')
        ax.set_xlabel('Number of required visits')
        ax.set_ylabel('Target count')


In [0]:
cmap = plt.get_cmap('tab10')

f = plt.figure(figsize=(8, 3), dpi=240)
gs = f.add_gridspec(1, 3, width_ratios=[2, 2, 3], wspace=0.4)

axs = [f.add_subplot(gs[0, 0]), f.add_subplot(gs[0, 1]), f.add_subplot(gs[0, 2], projection=wcs.wcs)]

for i, (k, target_list) in enumerate(target_lists.items()):
    if config.targets[k].prefix in ['sci']:
        if target_lists[k].has_diagram_values(cmd, observed=True):
            cmd.plot_observation(axs[0], target_list, c=target_lists[k].data['priority'],
                                 observed=True, cmap=cmap)

        if target_lists[k].has_diagram_values(ccd, observed=True):
            ccd.plot_observation(axs[1], target_list, c=target_lists[k].data['priority'], 
                                 observed=True, cmap=cmap)

        if target_lists[k].has_diagram_values(fov):
            l = fov.plot_observation(axs[2], target_list, c=target_lists[k].data['priority'], size=size[config.targets[k].prefix],
                                 observed=True, cmap=cmap)
            
pp = galaxy.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(axs[2], fov, corners=True, projection=SubaruWFC(p))

axs[2].set_aspect('equal', adjustable='datalim')
            
f.colorbar(l, ax=axs[2], label='target priority')

f.suptitle('Stars selected based on color cuts.')

In [0]:
f = plt.figure(figsize=(3, 3), dpi=240)
gs = f.add_gridspec(1, 1)

ax = f.add_subplot(gs[0, 0], projection=wcs.wcs)
for name, options in config.targets.items():
    if options.prefix in ['sky']:
        fov.plot_observation(ax, target_lists[name], c=colors[options.prefix], size=size[options.prefix], observed=True)

pp = galaxy.get_pointings(SubaruPFI)
for p in pp:
    pfi.plot_focal_plane(ax, fov, corners=True, projection=SubaruWFC(p))

ax.set_aspect('equal', adjustable='datalim')

f.suptitle('Available sky positions')